In [1]:
# Show flight path of a softball, baseball, golfball 
# For simplicity we assume flights start horizontally
# We ignore drag in the vertical direction.  

import pandas as pd

## pdaddrow is just a utility function for easy add of rows to a dataframe
def pdaddrow(indf,inline,dell=" "):
    import pandas as pd
    if type(inline) == str:
        inline=inline.split(dell)
    cols=indf.columns
    if len(cols) != len(inline):
        print("in pdaddrow row length mismatch dataframe = ",len(cols), " input row = ",len(inline))
        print(inline)
        if dell == "\t":
            print("delimiter is tab")
        else:
            print("delimiter is ",dell)
        return indf
    adict={}
    for (h,v) in zip(cols,inline) :
        adict[h]=[v]
    #print(adict)
    inline=pd.DataFrame(adict)
    return(pd.concat([indf,inline],ignore_index = True))
 


In [2]:
# Our ball descriptions
# Ball type
# Wt oz
# Initial speed ft/sec
# Width inches
# Drag coefficient 

sdat="""Ball	Ounce	FPS	width	drag
soft	6.0	58	3.5	.401
base	5.2	132	2.9	.401
golf	1.62	220	1.68	.395"""
dat=dat.split("\n")
header=dat[0].split()
tab=pd.DataFrame(columns=header)
for d in dat[1:] :
    tab=pdaddrow(tab,d,"\t")
tab

NameError: name 'dat' is not defined

In [ ]:
# Conversion functions
def grams(ounce):
    return(ounce*28.3495)

def cmps(fps):
    return(fps*12.0*2.54)

def mps(cps):
    return(cps/100.0)

def secpmile(fps):
    return(5280.0/fps)

def ergs(cmps,grams):
    return(0.5*grams*cmps**2)

def joules(ergs):
    return(ergs*1e-7)

def ftlb(joules):
    return(joules*0.737562)

def dm(fps):
    return((0.5*32*(5280.0/fps)**2))

def d50(fps):
    return((0.5*32*(150.0/fps)**2*12.0))

def pir2(d):
    from numpy import pi
    #convert from inches to 
    d=d*25.4
    r=d/2.0
    #retrun value is in m^2
    return((pi*r*r)/(1000**2))

In [ ]:
tab['Grams'] = tab.apply(lambda x: grams(float(x['Ounce'])), axis=1)
tab['cm/sec'] = tab.apply(lambda x: cmps(float(x['FPS'])), axis=1)
tab['m/sec'] = tab.apply(lambda x: mps(float(x['cm/sec'])), axis=1)
tab['sec/mile'] = tab.apply(lambda x: secpmile(float(x['FPS'])), axis=1)
tab['ergs'] = tab.apply(lambda x: ergs(float(x['cm/sec']),float(x['Grams'])), axis=1)
tab['Joules'] = tab.apply(lambda x: joules(float(x['ergs'])), axis=1)
tab['Ft-lbs'] = tab.apply(lambda x: ftlb(float(x['Joules'])), axis=1)
tab['mile drop (ft)'] = tab.apply(lambda x: dm(float(x['FPS'])), axis=1)
tab['50 yd drop (in)'] = tab.apply(lambda x: d50(float(x['FPS'])), axis=1)
tab['area'] = tab.apply(lambda x: pir2(float(x['width'])), axis=1)


In [ ]:
tab

In [ ]:
# Numerical solution to differential equations for a projectile with drag in horizontal direction
# Drag is ignored in the vertical direction, would be easy to add.
def xvt(area=4.8e-5,c=0.295,mass=0.016,v0=400,tmax=1,dt=0.01):
    import numpy as np
    from scipy.integrate import odeint
    def drag(v):
        p=1.2
        f=-(c*p*area*v*v)/2.0
        #f=-(c*p*a*400*400)/2.0
        return(f)
    def dvdt(y,t):
        acc=drag(y[1])/mass
        return(y[1],acc)
    y0=np.array([0,v0])
    #print(y0)
    a=0
    b=tmax+dt
    t=np.arange(a,b,dt)
    y = odeint(dvdt, y0, t)
    #print(y)
    arr = np.empty((0,2), float)
    #print(arr)
    for item in zip(t,y):
        #print(item[0],item[1][0])
        #print(item[0],item[1][0])
        arr = np.append(arr, np.array([[item[0],item[1][0]]]), axis=0)
        #arr = np.append(arr,z, axis=0)
    v = np.empty((0,2), float)
    #print(arr)
    for item in zip(t,y):
        #print(item[0],item[1][0])
        #print(item[0],item[1][0])
        v = np.append(v, np.array([[item[0],item[1][1]]]), axis=0)
        #arr = np.append(arr,z, axis=0)
    return(arr,v)


def zt(tmax=1,dt=0.01):
    import numpy as np
    from scipy.integrate import odeint
    def grav(y,t):
        return(y[1],-9.8)
    y0=np.array([0,0])
    #print(y0)
    a=0
    b=tmax+dt
    t=np.arange(a,b,dt)
    y = odeint(grav, y0, t)
    arr = np.empty((0,2), float)
    #print(arr)
    for item in zip(t,y):
        #print(item[0],item[1][0])
        #print(item[0],item[1][0])
        arr = np.append(arr, np.array([[item[0],item[1][0]]]), axis=0)
        #arr = np.append(arr,z, axis=0)
    return(arr)



In [ ]:
# Solve for each ball type
tt=5.0
delta=0.005

ball="soft"
case=tab.loc[tab['Ball']==ball]
v=float(case['m/sec'])
area=float(case['area'])
mass=float(case['Grams']/1000.0)
drag=float(case['drag'])
print(ball,v,area,mass)
(xrangesoft,vsoft)=xvt(area=area,mass=mass,v0=v,tmax=tt,dt=delta,c=drag)
dropsoft=zt(tmax=tt,dt=delta)

ball="base"
case=tab.loc[tab['Ball']==ball]
v=float(case['m/sec'])
area=float(case['area'])
mass=float(case['Grams']/1000.0)
drag=float(case['drag'])

print(ball,v,area,mass)
(xrangebase,vbase)=xvt(area=area,mass=mass,v0=v,tmax=tt,dt=delta,c=drag)
dropbase=zt(tmax=tt,dt=delta)

ball="golf"
case=tab.loc[tab['Ball']==ball]
v=float(case['m/sec'])
area=float(case['area'])
mass=float(case['Grams']/1000.0)
drag=float(case['drag'])

print(ball,v,area,mass)
(xrangegolf,vgolf)=xvt(area=area,mass=mass,v0=v,tmax=tt,dt=delta,c=drag)
dropgolf=zt(tmax=tt,dt=delta)


In [ ]:
# Put in a data frame
tab=pd.DataFrame(columns=header)
header=["Time","Softball-x","Baseball-x","Golf-x","Softball-v","Baseball-v","Golf-v"] 
output=pd.DataFrame(columns=header)
for l in range(0,len(xrangesoft)) :
    line=[xrangesoft[l][0],xrangesoft[l][1],xrangebase[l][1],xrangegolf[l][1],vsoft[l][1],vbase[l][1],vgolf[l][1]]
    output=pdaddrow(output,line)

output

In [ ]:
# do some plots
p=output.plot(x='Time',y=['Softball-x','Baseball-x','Golf-x'],title="Ball Range",xlabel="Seconds",ylabel="Meters",figsize=[11*.75,8.5*.75])
p
fig=p.get_figure()

In [ ]:
p=output.plot(x='Time',y=['Softball-v','Baseball-v','Golf-v'],title="Ball Speed",xlabel="Seconds",ylabel="Meters/Second",figsize=[11*.75,8.5*.75])
p
fig=p.get_figure()